In [1]:
import numpy as np
from scipy.optimize import root_scalar
import sympy as sp
import time
from tqdm import tqdm

# from scipy.optimize import fsolve

# # LUCB algorithm

# num_sim = 500 # for stopping time testing
num_sim = 100 # for runtime testing
num_arm = 4

total_run_time_array = np.zeros(num_sim)

def arm_selector(S_list, alpha):
  n = len(S_list)
  upp_bounds = np.ones(n)
  mus = np.ones(n) * 0.5

  for i in range(n):
    mus[i] = np.mean(S_list[i])
    t = len(S_list[i])
    rad = np.sqrt(np.log(405.5 * n * t ** (1.1) / alpha * np.log(405.5 * n * t ** (1.1) / alpha) ) / (2*t) )
    upp_bounds[i] = mus[i] + rad

  arm_1 = np.argmax(mus)
  upp_bounds[arm_1] = -1E10
  arm_2 = np.argmax(upp_bounds)

  return (arm_1, arm_2)


def e_val(S_list, mus, m):
  val = 0
  for i in range(4):
    val +=  0.25 * np.prod(1 + (np.array(S_list[i]) - np.asarray(m[i])) * (np.array(mus[i]) - np.asarray(m[i])) / 0.26)
  return val


solver= 'brenth'

# simulation for best arm identification using our approach
best_arm_list = []
stop_times = []

#mu = np.arange(4) + 1
#mu = (0.71 - 0.14*(mu - 1))/(0.29 + 0.14*(mu - 1))

mu = [0.29, 0.43, 0.57, 0.71]
horizon_len = 1000 # this is really 2000 samples (i.e. 1000 LUCB iterations)
# mu = [0.01, 0.011, 0.009, 0.013]

for j in tqdm(range(num_sim)):
  partitions = np.zeros(4)
    
  # set seed within simulation
  np.random.seed(j)
    
  S_list = [np.random.binomial(1, p, 1) for p in mu] # the last 1 represents the size
  # S_list = [[np.random.beta(1, p, 1)] for p in mu] # the last 1 represents the size
  mus = [[0.5]] * num_arm
  alpha = 0.05
  t = 4

  for i in range(horizon_len): # for runtime testing
  #while (np.sum(partitions)<3): # for stopping time testing
    which_arm = arm_selector(S_list, alpha)
    h = which_arm[0]
    l = which_arm[1]

    # update mus
    mus[h] = np.append(mus[h], [((1/2 + sum(S_list[h])) / (len(S_list[h])  + 1))])
    # print((1/2 + sum(S_list[h])) / (len(S_list[h])  + 1))
    mus[l] = np.append(mus[l], [((1/2 + sum(S_list[l])) / (len(S_list[l])  + 1))])
    # mus[l] = np.asarray(mus[l])

    # update samples
    # S_list[h] = np.append(S_list[h], np.random.beta(1, mu[h], 1))
    S_list[h] = np.append(S_list[h], np.random.binomial(1, mu[h], 1))
    # S_list[h] = np.asarray(S_list[h])
    # S_list[l] = np.append(S_list[l], np.random.beta(1, mu[l], 1))
    S_list[l] = np.append(S_list[l], np.random.binomial(1, mu[l], 1))
    # S_list[l] = np.asarray(S_list[l])

    ######### start timing
    start_time = time.time()


    # find the global minima in our set
    min_values = np.zeros(num_arm)
    coeffs = np.zeros(num_arm)
    for i in range(num_arm):
      S = np.asarray(S_list[i])
      mu_hat = np.asarray(mus[i])
      def equation(x):
        return sum((2*x-mu_hat-S)/(0.26 + (S-x)*(mu_hat-x)))
      root = root_scalar(equation, method=solver, bracket=[0,1], xtol=0.1**10)
      # print(root)
      min_values[i] = root.root

    # determine which arm is currently best
    ranks = np.argsort(min_values)[::-1] # Get the indices that would sort the array in descending order

    best_arm = ranks[0]

    #print(min_values)

    # minimum e-value globally corresponds to partition with best arm.
    test_values = np.zeros(num_arm)
    test_values[best_arm] = e_val(S_list, mus, min_values)

    # for the second best arm, we only need to project second best, best
    second_best = ranks[1]

    # get coefficient function
    def coeffs(x, S_list, mus, a):
      return np.prod(1/0.26* (0.26 + (np.asarray(S_list[a], dtype=np.float64) - x) * (np.asarray(mus[a], dtype=np.float64)-x ) ))

    def equation2(x): # we want to do element wise subtraction from x
      return coeffs(x, S_list, mus, best_arm) *  sum((2*x-np.asarray(mus[best_arm])-np.asarray(S_list[best_arm]))/(0.26 + (np.asarray(S_list[best_arm])-x)*(np.asarray(mus[best_arm])-x))) + \
      coeffs(x, S_list, mus, second_best) * sum((2*x-np.asarray(mus[second_best])-np.asarray(S_list[second_best]))/(0.26 + (np.asarray(S_list[second_best])-x)*(np.asarray(mus[second_best])-x)))
    val = root_scalar(equation2, method=solver, bracket=[0,1], xtol=0.1**10)
    temp = min_values.copy()
    temp[best_arm] = np.minimum(val.root, min_values[best_arm])
    temp[second_best] = val.root
    #print(temp)
    test_values[second_best] = e_val(S_list, mus, temp)


    # for the third best arm, we need to project third best, second best, best
    third_best = ranks[2]
    def equation3(x):
      return coeffs(x, S_list, mus, best_arm) * sum((2*x-np.asarray(mus[best_arm])-np.asarray(S_list[best_arm]))/(0.26 + (np.asarray(S_list[best_arm])-x)*(np.asarray(mus[best_arm])-x))) + \
       coeffs(x, S_list, mus, second_best) * (x < min_values[second_best]) *  sum((2*x-np.asarray(mus[second_best])-np.asarray(S_list[second_best]))/(0.26 + (np.asarray(S_list[second_best])-x)*(np.asarray(mus[second_best])-x))) + \
        coeffs(x, S_list, mus, third_best) * sum((2*x-np.asarray(mus[third_best])-np.asarray(S_list[third_best]))/(0.26 + (np.asarray(S_list[third_best])-x)*(np.asarray(mus[third_best])-x)))
    val = root_scalar(equation3, method=solver, bracket=[0,1], xtol=0.1**5)

    temp = min_values.copy()
    temp[best_arm] = np.minimum(val.root, min_values[best_arm])
    temp[second_best] = np.minimum(val.root, min_values[second_best])
    temp[third_best] = val.root
    #print(temp)
    test_values[third_best] = e_val(S_list, mus, temp)

    # for the fourth best arm, we project all values
    fourth_best = ranks[3]
    def equation4(x):
      return coeffs(x, S_list, mus, best_arm) * sum((2*x-np.asarray(mus[best_arm])-np.asarray(S_list[best_arm]))/(0.26 + (np.asarray(S_list[best_arm])-x) * (np.asarray(mus[best_arm])-x))) + \
        coeffs(x, S_list, mus, second_best) * (x < min_values[second_best]) *  sum((2*x-np.asarray(mus[second_best])-np.asarray(S_list[second_best]))/(0.26 + (np.asarray(S_list[second_best])-x)*(np.asarray(mus[second_best])-x))) + \
        coeffs(x, S_list, mus, third_best) * (x < min_values[third_best]) * sum((2*x-np.asarray(mus[third_best])-np.asarray(S_list[third_best]))/(0.26 + (np.asarray(S_list[third_best])-x)*(np.asarray(mus[third_best])-x))) + \
        coeffs(x, S_list, mus, fourth_best) * sum((2*x-np.asarray(mus[fourth_best])-np.asarray(S_list[fourth_best]))/(0.26 + (np.asarray(S_list[fourth_best])-x)*(np.asarray(mus[fourth_best])-x)))
    val = root_scalar(equation4, method=solver, bracket=[0,1], xtol=0.1**10)

    temp = min_values.copy()
    temp[best_arm] = np.minimum(val.root, min_values[best_arm])
    temp[second_best] = np.minimum(val.root, min_values[second_best])
    temp[third_best] = np.minimum(val.root, min_values[third_best])
    temp[fourth_best] = val.root
    #print(temp)
    test_values[fourth_best] = e_val(S_list, mus, temp)
    # print(test_values)
    #print(test_values)

    #check to see which arms can be eliminated
    reject = (test_values >= 1/alpha)
    # print(reject)
    partitions = np.maximum(reject, partitions)

    end_time = time.time()
    total_run_time_array[j] += end_time - start_time

    ########### end timing

    # update the number of pulls
    t += 2


  best_arm_list.append(np.where(partitions==0)[0][0])
  stop_times.append(t)
  print(t)


print(np.mean(total_run_time_array))
print(np.std(total_run_time_array))

#   print(stop_times)
#   print(best_arm_list)

print(np.mean(stop_times))
print(np.std(stop_times))




  0%|                                                   | 0/100 [00:00<?, ?it/s]/var/folders/b0/dg8r3zxn7d3g6xzk1gk57mrc0000gn/T/ipykernel_56608/774313837.py:122: RuntimeWarning: overflow encountered in scalar multiply
  return coeffs(x, S_list, mus, best_arm) *  sum((2*x-np.asarray(mus[best_arm])-np.asarray(S_list[best_arm]))/(0.26 + (np.asarray(S_list[best_arm])-x)*(np.asarray(mus[best_arm])-x))) + \
/var/folders/b0/dg8r3zxn7d3g6xzk1gk57mrc0000gn/T/ipykernel_56608/774313837.py:135: RuntimeWarning: overflow encountered in scalar multiply
  return coeffs(x, S_list, mus, best_arm) * sum((2*x-np.asarray(mus[best_arm])-np.asarray(S_list[best_arm]))/(0.26 + (np.asarray(S_list[best_arm])-x)*(np.asarray(mus[best_arm])-x))) + \
/var/folders/b0/dg8r3zxn7d3g6xzk1gk57mrc0000gn/T/ipykernel_56608/774313837.py:150: RuntimeWarning: overflow encountered in scalar multiply
  return coeffs(x, S_list, mus, best_arm) * sum((2*x-np.asarray(mus[best_arm])-np.asarray(S_list[best_arm]))/(0.26 + (np.asarray(S

2004


  2%|▊                                          | 2/100 [00:10<08:34,  5.25s/it]

2004


  3%|█▎                                         | 3/100 [00:15<08:39,  5.36s/it]

...

2004


 99%|█████████████████████████████████████████▌| 99/100 [08:43<00:04,  4.96s/it]

2004


100%|█████████████████████████████████████████| 100/100 [08:48<00:00,  5.28s/it]

2004
5.212039794921875
0.22697138652251098
2004.0
0.0


In [17]:
# for stoppping times
print(np.mean(stop_times))
print(np.std(stop_times))
print(np.mean(np.equal(best_arm_list, 3)))


692.04
238.42650523798733
1.0


In [2]:
# for runtime testing\
print(np.mean(total_run_time_array))
print(np.std(total_run_time_array))

5.212039794921875
0.22697138652251098
